In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


# Task: Columns -> Energy & Petrol Consumption, Storage

In [182]:
base_url = "https://www.fueleconomy.gov/feg/"
makers = ["Chevrolet", "Ford", "Nissan"]

def get_data(base, maker):
    mpge = []
    mpg = []
    battery = []
    gas_tank = []
    for m in maker:
        page = 1
        while True:
            print(f"Scraping page {page} of {m}")
            maker_url = f"{base}PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make={m}&srchtyp=newMake&pageno={page}&rowLimit=50"
            response = r.get(maker_url)
            soup = bs(response.content)
                
            link = [x.get("href") for x in soup.select("a[href]") if "sbs&id" in x.get("href")]
            if len(link) == 0:
                break
                
            for l in link:
                if l == "#":
                    continue
                response = r.get(base+l)
                soup = bs(response.content)
                try:
                    type = soup.find("td", class_="fuel nobottomborder padding").getText(strip=True)
                except:
                    print(f"No type: {l}")
                try:
                    if type == "Electricity":
                        battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                        gas_tank.append(np.nan)
                        mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                        mpg.append(np.nan)
                    elif type in ["Regular Gasoline", "Diesel", "Premium Gasoline"]:
                        battery.append(np.nan)
                        gas_tank.append(soup.find_all("td", class_="sbsEconData")[2].getText().split(" ")[0]) #gallons
                        mpge.append(np.nan)
                        mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                    else:
                        battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                        gas_tank.append(soup.find_all("td", class_="sbsEconData")[3].getText().split(" ")[0]) #gallons
                        mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                        mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                except:
                    print(f"Process stopped at {l}")
            page += 1

    print("Creating Dictionary.")
    manufacturer_dict = {"electric_consumption":mpge, "electric_storage":battery,
                        "analogue_consumption":mpg, "analogue_storage":gas_tank}
    print("Returning Dataframe.")
    return pd.DataFrame(manufacturer_dict)

manufacturer_df = get_data(base_url, makers)
print("Finished")

Scraping page 1 of Chevrolet
Scraping page 2 of Chevrolet
Scraping page 3 of Chevrolet
Scraping page 4 of Chevrolet
Scraping page 5 of Chevrolet
Scraping page 6 of Chevrolet
Scraping page 7 of Chevrolet
Scraping page 8 of Chevrolet
Scraping page 9 of Chevrolet
Scraping page 10 of Chevrolet
Scraping page 11 of Chevrolet
Scraping page 12 of Chevrolet
Scraping page 1 of Ford
Scraping page 2 of Ford
Scraping page 3 of Ford
Scraping page 4 of Ford
Scraping page 5 of Ford
Scraping page 6 of Ford
Scraping page 7 of Ford
Scraping page 8 of Ford
Scraping page 9 of Ford
Scraping page 10 of Ford
Scraping page 11 of Ford
Scraping page 12 of Ford
Scraping page 1 of Nissan
Scraping page 2 of Nissan
Scraping page 3 of Nissan
Scraping page 4 of Nissan
Scraping page 5 of Nissan
Scraping page 6 of Nissan
Creating Dictionary.
Returning Dataframe.
Finished


In [ ]:
manufacturer_df.describ

maker_url = f"https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&path=4&year1=2023&year2=2025&make=Chevrolet&srchtyp=newMake&pageno=4&rowLimit=50&sortBy=Comb&tabView=0"
response = r.get(maker_url)
soup = bs(response.content)
                
link = [x.get("href") for x in soup.select("a[href]") if "sbs&id" in x.get("href")]
print(link)